In [1]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re

api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
issues = api.download_issues(project_id="pbssoftware")
filtered = [i for i in issues.values() if i.sprint_milestone.name == "3.0"]


magic = re.compile(r"[\+\*]+(3.0Web)?(FRS\d+[0-9\.]+)")
magic2 = re.compile(r"(3.0Web)?(FRS\d+[0-9\.]+)")

In [ ]:
frs = set()
frs2 = set()

for v in filtered:
    d = v.description or ""
    l = magic.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs.add(result)
    l = magic2.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs2.add(result)
    
sorted(frs)

In [ ]:
class FRSItem():
    def __init__(self, issue, levels):
        self.issue = issue
        self.levels = levels
    
    @classmethod
    def from_str(cls, s):
        id, *nums = s.split(".")
        issue = id[3:]
        print(s, id, nums)
        issue = int(issue)
        level = [int(n) for n in nums]
        return cls(issue, level)
    
    def is_child(self, parent):
        if len(parent.level) >= len(self.level):
            return False
        for c, p in zip(self.level, parent.level):
            if c != p:
                return False
            
        # Don't need to check for parent level == child level 
        # already checked for matching level + matching level len
        return True
        
    def is_parent(self, child):
        if len(self.level) >= len(child.level):
               return False
        for c, p in zip(child.level, self.level):
               if c != p:
                   return False
        return True
            

In [ ]:
class Failure(AssertionError):
    pass

def assert_(exp, msg=None, *args):
    if not exp:
        if msg and args:
            msg = msg % args
        raise Failure(msg)
    
def check_regex(m, exp):
    assert_(m, "No match found")
    assert_(m == exp, "%r != %r", m, exp)

def check_frs_findall(s, exp):
    check_regex(frs_findall(s), exp)

check_frs_findall("*FRS1234*", ["FRS1234"])
check_frs_findall("*FRS1234.5*", ["FRS1234.5"])
check_frs_findall("*FRS1234.5.6", ["FRS1234.5.6"])
check_frs_findall("* *FRS1234.5.6*", ["FRS1234.5.6"])
check_frs_findall("*FRS2851.0.3* Time stamps for edge cases during generation by batch:", 
                  ["FRS2851.0.3"])

In [ ]:
frs = []
frs_re = re.compile(r"^(?:[\*\# ]+)?(FRS\d+[0-9\.]+)")
frs_match = frs_re.match
frs_findall = frs_re.findall

for v in filtered:
    d = v.description or ""
    l = frs_findall(d)
    for f in l:
        frs.append(f[1])
len(frs)
print(frs)
len(filtered), len(frs)

In [ ]:
FRSItem.from_str(frs[20])

In [ ]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
try:
    cache
except NameError:
    cache = None
cache = cache or {}
magic = re.compile(r"[\+\*\s]*(3.0Web)?(FRS\d+[0-9\.]+)[\+\*: \-]*")

def get_issues():
    global cache
    if not cache:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        iss = api.download_issues(project_id="pbssoftware")
        cache = iss.copy()
    else:
        iss = cache.copy()
    return iss

def extract_frs_items2():
    iss = get_issues()
    filtered = [i for i in iss.values() if i.sprint_milestone.name == "3.0"]
    reqs = []
    frs = set()
    for v in filtered:
        d = v.description or ""
        lines = d.split("\r\n")
        i = 0
        while i < len(lines):
            l = lines[i]
            m = magic.search(l)
            if m:
                web, fr = m.groups()
                web = web or ""
                rest = l[m.end():]
                other = []
                i += 1
                while i < len(lines):
                    l = lines[i]
                    if not l.strip():
                        break
                    elif magic.search(l):
                        break
                    else:
                        other.append(l.strip())
                    i += 1                        
                result = (v.id, web + fr, rest, "\n".join(other))
                reqs.append(result)
            i += 1    
    reqs.sort()
    return reqs

reqs= extract_frs_items2()
with open("test.txt", 'w') as f:
    for id, fr, rest, other in reqs:
        f.write("*%s*%s\n"%(fr, ": %s" % rest if rest else ""))
        f.write(other)
        f.write("\n\n")
import os; os.startfile(f.name)

In [1]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
try:
    cache
except NameError:
    cache = {}
magic = re.compile(r"[\+\*\s]*(3.0Web)?(FRS\d+[0-9\.]+)[\+\*: \-]*")

def get_issues():
    global cache
    if not cache:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        iss = api.download_issues(project_id="pbssoftware")
        cache = iss
    else:
        iss = cache
    return [i for i in iss.values() if i.sprint_milestone.name == "3.0"]

def extract_frs_items3():
    filtered = get_issues()
    reqs = []
    for v in filtered:
        d = v.description or ""
        if magic.search(d):
            reqs.append((v.id, d.strip().replace("\r\n", "\n")))
    reqs.sort()
    return reqs

# reqs= extract_frs_items3()
# with open("compiled_frs_descriptions.txt", 'w', newline="\r\n") as f:
#     for id, desc in reqs:
#         f.write("+*Issue %d:*+\n" % id)
#         f.write(desc)
#         f.write("\n\n")
# import os; os.startfile(f.name)

In [2]:
def extract_word_items(word="login", exclude=()):
    filtered = get_issues()
    exclude = frozenset(exclude)
    logins = []
    for v in filtered:
        if v.id in exclude: continue
        if magic.search(v.description) and word in v.description.lower():
            logins.append((v.id, v.description.strip().replace("\r\n", "\n")))
    logins.sort()
    return logins

exclude = 1073, 2828, 1759, 2367, 2445, 2878, 2881, 2911, 3077
logins = extract_word_items("login", exclude)
def write_word_items(word, items):
    with open("%s_issues.txt" % word, 'w') as f:
        for id, desc in items:
            f.write("+*Issue %d:*+\n" % id)
            f.write(desc)
            f.write("\n\n")
    return f.name
#import os; os.startfile(write_word_items("login", logins))

In [3]:
def extract(word):
    import os; os.startfile(write_word_items(word, extract_word_items(word)))

In [ ]:
import os; os.startfile(write_word_items("permission", extract_word_items("permission")))

In [ ]:
extract("user events")

In [ ]:
extract("midnight")

In [6]:
extract("graph")

In [7]:
extract("database")

In [4]:
extract("unknown user")